In [2]:

import splitfolders

splitfolders.ratio("flowers", # The location of dataset
                   output="FlowerDatasetDivided", # The output location
                   seed=42, # The number of seed
                   ratio=(.8, .2, ), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )

In [47]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [48]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
training_set = train_datagen.flow_from_directory("FlowerDatasetDivided/train",
                                                target_size = (64,64),
                                                batch_size= 32,
                                                classes = ['daisy','dandelion','rose','sunflower','tulip'],
                                                class_mode="categorical")

Found 3452 images belonging to 5 classes.


In [49]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory("FlowerDatasetDivided/test",
                                           target_size = (64,64),
                                           batch_size = 32,
                                           classes = ['daisy','dandelion','rose','sunflower','tulip'],
                                           class_mode="categorical")

Found 865 images belonging to 5 classes.


In [50]:
cnn = tf.keras.models.Sequential()

In [51]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64,64,3]))

In [52]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [53]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [54]:
cnn.add(tf.keras.layers.Flatten())

In [55]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [56]:
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))

In [57]:
from tensorflow.keras.optimizers import RMSprop
cnn.compile(optimizer=RMSprop(lr=0.001), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [58]:
cnn.fit(x = training_set, validation_data=test_set, epochs = 30)

Epoch 1/30
108/108 [==============================] - 9s 75ms/step - loss: 1.3375 - accuracy: 0.4351 - val_loss: 1.1242 - val_accuracy: 0.5387
Epoch 2/30
108/108 [==============================] - 8s 74ms/step - loss: 1.1283 - accuracy: 0.5545 - val_loss: 1.0817 - val_accuracy: 0.5584
Epoch 3/30
108/108 [==============================] - 8s 73ms/step - loss: 1.0157 - accuracy: 0.6034 - val_loss: 0.9674 - val_accuracy: 0.6000
Epoch 4/30
108/108 [==============================] - 8s 75ms/step - loss: 0.9483 - accuracy: 0.6283 - val_loss: 0.9541 - val_accuracy: 0.6289
Epoch 5/30
108/108 [==============================] - 8s 76ms/step - loss: 0.8808 - accuracy: 0.6677 - val_loss: 0.8884 - val_accuracy: 0.6532
Epoch 6/30
108/108 [==============================] - 8s 77ms/step - loss: 0.8222 - accuracy: 0.6897 - val_loss: 0.8137 - val_accuracy: 0.6832
Epoch 7/30
108/108 [==============================] - 8s 75ms/step - loss: 0.7858 - accuracy: 0.6958 - val_loss: 0.9249 - val_accuracy: 0.6509

In [59]:
cnn.save("model1.h5")

In [69]:
import numpy as np
import keras.utils as image
test_image = image.load_img('tulipTestImage.jpg', target_size = (64,64))
#test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
print(result)
if result[0][0] == 1:
    print("DAISY")
elif result[0][1] == 1:
    print("DANDELION")
elif result[0][2] == 1:
    print("ROSE")
elif result[0][3] == 1:
    print("SUNFLOWER")
else:
    print("TULIP")

1/1 [==============================] - 0s 15ms/step
[[0. 0. 0. 0. 1.]]
TULIP
